In [1]:
import numpy as np
import pandas as pd
import pdfplumber 
from openpyxl import load_workbook

## CREATING DUMMY DATAFRAMES FOR COMPARISION

In [2]:
## ORIGINAL REPORTS WHICH ARE ALREADY PRESENT IN THE SOURCE SYSTEM
ORGINAL_REPORT_DICT = {
    'MODEL' : ['MDL_1','MDL_2','MDL_3','MDL_4','MDL_5','MDL_6'],
    'DAY_COUNT' : [13,20,45,87,27,0],
    'MONTH_TILL_TODAY_COUNT' : [123,203,453,234,657,9],
    'LAST_YR_SAME_DAY' :  [16,19,42,80,22,10]
}

# RECREATED REPORTS WHICH ARE RECREATED DUE TO SOME SYSTEM FAILURE OR SERVER CRASH USING DEFINED LOGIC
RECREATED_REPORT_DICT = {
    'MODEL' : ['MDL_1','MDL_2','MDL_3','MDL_4','MDL_5','MDL_6'],
    'DAY_COUNT' : [13,20,45,87,27,0],
    'MONTH_TILL_TODAY_COUNT' : [123,203,453,234,657,9],
    'LAST_YR_SAME_DAY' :  [16,19,42,80,22,10]
}

## CREATING ORIGINAL DATAFRAME USING ORIGINAL DATA DICTIONARY

In [3]:
ORIGINAL_RPRT_DF = pd.DataFrame(ORGINAL_REPORT_DICT)
ORIGINAL_RPRT_DF.head()

,MODEL,DAY_COUNT,MONTH_TILL_TODAY_COUNT,LAST_YR_SAME_DAY
0,MDL_1,13,123,16
1,MDL_2,20,203,19
2,MDL_3,45,453,42
3,MDL_4,87,234,80
4,MDL_5,27,657,22


## CREATING RECREATED DATAFRAME USING RECREATED DATA DICTIONARY

In [4]:
RECREATED_RPRT_DF = pd.DataFrame(RECREATED_REPORT_DICT)
RECREATED_RPRT_DF.head()

,MODEL,DAY_COUNT,MONTH_TILL_TODAY_COUNT,LAST_YR_SAME_DAY
0,MDL_1,13,123,16
1,MDL_2,20,203,19
2,MDL_3,45,453,42
3,MDL_4,87,234,80
4,MDL_5,27,657,22


## CLEAN THE DATA BEFORE COMPARISION (Dealing with null or Na Values )

In [5]:
## COMMON COLUMN IN THE BOTH DATAFRAMES 
COMMON_COLUMN_LIST = ['MODEL']

for col in COMMON_COLUMN_LIST:
    ORIGINAL_RPRT_DF[col] = ORIGINAL_RPRT_DF[col].str.upper() ## CONVERTING TO UPPER CASE TO AVOID IF IS THERE CASE SENSITIVE MODELS 
    RECREATED_RPRT_DF[col] = RECREATED_RPRT_DF[col].str.upper()

ASCENDIN_ORDER_STRING = [True for col in COMMON_COLUMN_LIST]

ORIGINAL_RPRT_DF.sort_values(COMMON_COLUMN_LIST,ascending=ASCENDIN_ORDER_STRING,inplace=True)
ORIGINAL_RPRT_DF.reset_index(drop=True)

RECREATED_RPRT_DF.sort_values(COMMON_COLUMN_LIST,ascending=ASCENDIN_ORDER_STRING,inplace=True)
RECREATED_RPRT_DF.reset_index(drop=True)

,MODEL,DAY_COUNT,MONTH_TILL_TODAY_COUNT,LAST_YR_SAME_DAY
0,MDL_1,13,123,16
1,MDL_2,20,203,19
2,MDL_3,45,453,42
3,MDL_4,87,234,80
4,MDL_5,27,657,22
5,MDL_6,0,9,10


In [6]:
ORIGINAL_RPRT_DF.head()

,MODEL,DAY_COUNT,MONTH_TILL_TODAY_COUNT,LAST_YR_SAME_DAY
0,MDL_1,13,123,16
1,MDL_2,20,203,19
2,MDL_3,45,453,42
3,MDL_4,87,234,80
4,MDL_5,27,657,22


In [7]:
RECREATED_RPRT_DF.head()

,MODEL,DAY_COUNT,MONTH_TILL_TODAY_COUNT,LAST_YR_SAME_DAY
0,MDL_1,13,123,16
1,MDL_2,20,203,19
2,MDL_3,45,453,42
3,MDL_4,87,234,80
4,MDL_5,27,657,22


## creating comparision df so that we can have single dataframe which can be used to get comparision final df

In [8]:
COMPARISION_DF = pd.merge(
    ORIGINAL_RPRT_DF,RECREATED_RPRT_DF,
    on=COMMON_COLUMN_LIST,
    how='outer',
    suffixes=['_ORIGINAL','_RECREATED'] ## providing suffixes for the columns so that we can understand from which dataframe the column is coming
)

## COMPARISON DATAFRAME

In [9]:
COMPARISION_DF.head()

,MODEL,DAY_COUNT_ORIGINAL,MONTH_TILL_TODAY_COUNT_ORIGINAL,LAST_YR_SAME_DAY_ORIGINAL,DAY_COUNT_RECREATED,MONTH_TILL_TODAY_COUNT_RECREATED,LAST_YR_SAME_DAY_RECREATED
0,MDL_1,13,123,16,13,123,16
1,MDL_2,20,203,19,20,203,19
2,MDL_3,45,453,42,45,453,42
3,MDL_4,87,234,80,87,234,80
4,MDL_5,27,657,22,27,657,22


#### To create Final comparision df we are getting all colomns that are needed so that we can arrange them properly side by side

In [10]:
COMPARISION_COLUMNS = COMPARISION_DF.filter(regex='_ORIGINAL$').columns.str.replace('_ORIGINAL', '')
COMPARISION_COLUMNS

Index(['DAY_COUNT', 'MONTH_TILL_TODAY_COUNT', 'LAST_YR_SAME_DAY'], dtype='object')

## Intial creation of Final comparision Dataframe

In [11]:
FINAL_COMPARISION_DF = COMPARISION_DF[COMMON_COLUMN_LIST].copy()
FINAL_COMPARISION_DF.head()

,MODEL
0,MDL_1
1,MDL_2
2,MDL_3
3,MDL_4
4,MDL_5


In [12]:
for col in COMPARISION_COLUMNS:
    org_col = col + '_ORIGINAL'
    recreated_col = col + '_RECREATED'
    diff_col = col + '_DIFF'
    FINAL_COMPARISION_DF[org_col] = COMPARISION_DF[org_col]
    FINAL_COMPARISION_DF[recreated_col] = COMPARISION_DF[recreated_col]
    FINAL_COMPARISION_DF[diff_col] = COMPARISION_DF[org_col].fillna(0) - COMPARISION_DF[recreated_col].fillna(0)

## FINAL COMPARTSION DATAFRAME
FINAL_COMPARISION_DF.head()

,MODEL,DAY_COUNT_ORIGINAL,DAY_COUNT_RECREATED,DAY_COUNT_DIFF,MONTH_TILL_TODAY_COUNT_ORIGINAL,MONTH_TILL_TODAY_COUNT_RECREATED,MONTH_TILL_TODAY_COUNT_DIFF,LAST_YR_SAME_DAY_ORIGINAL,LAST_YR_SAME_DAY_RECREATED,LAST_YR_SAME_DAY_DIFF
0,MDL_1,13,13,0,123,123,0,16,16,0
1,MDL_2,20,20,0,203,203,0,19,19,0
2,MDL_3,45,45,0,453,453,0,42,42,0
3,MDL_4,87,87,0,234,234,0,80,80,0
4,MDL_5,27,27,0,657,657,0,22,22,0


#### Converting Final Comparision Dataframe to Excel so that business team can review this Excel as it is Easy to understand

In [13]:
FINAL_COMPARISION_DF.to_excel('NEW_LEARNING_EXCEL_FILE.xlsx',index=False)